<a href="https://colab.research.google.com/github/descartesmbogning/Marburg-virus-research/blob/main/3_analysis_number_of_authors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
from functools import reduce
import seaborn as sns
import missingno as msno
%matplotlib inline

import matplotlib.pyplot as plt
from scipy.stats import norm


In [ ]:
# Set the figure size - handy for larger output
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = [10, 6]
# Set up with a higher resolution screen (useful on Mac)
%config InlineBackend.figure_format = 'retina'

In [ ]:
from scipy import stats #the only import you actually need for this
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}

In [ ]:
# function that tests distribution then picks test accordingly
def test_sig(x, y):
    if stats.shapiro(x).pvalue >= 0.05 and stats.shapiro(y).pvalue >= 0.05: #if they are normally distriuted
        print("t-test")
        return stats.ttest_ind(x,y)
    else:
        print("mann-whitney")
        return stats.mannwhitneyu(x, y)

## import data

In [ ]:
data = pd.read_csv('D:\MARBURG VIRUS DISEASES\TABLES/data_fund_region_country_and_org_clean_dummies_col_replace_grp.csv')#.drop(['Addresses'], axis=1)
data = data.loc[data['funding_yes'] == 1]
data.head(2)

,Unnamed: 0,wos_ID,funding_yes,open_access_yes,Funding Name Preferred,europe and central asia,north america,africa,sub-saharan africa,east asia and pacific,...,fulbright foundation - usa,world bank - multilateral,china association for science and technology - china,zhejiang university - china,Funding Orgs,Open Access Designations,"Times Cited, All Databases",180 Day Usage Count,Since 2013 Usage Count,Publication Year
1,1,wos:000921279800001,1,1,National Institutes of Health (NIH)(United Sta...,NaN,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,national institutes of health (nih) [u19ai1427...,hybrid,NaN,NaN,NaN,2023.0
2,2,wos:000909001200001,1,1,Russian Science Foundation(Russian Science Fou...,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,russian science foundation; [22-13-00036],"gold, green accepted",NaN,3.0,3.0,2023.0


In [ ]:
country_data = pd.read_pickle('D:\MARBURG VIRUS DISEASES\DATA_PREPROCESSING/countries_split.pkl').drop('Addresses', axis=1)
country_data.head(1)

,Afghanistan,United States,Angola,Argentina,Australia,Austria,Bangladesh,Belgium,Belize,Brazil,...,Thailand,Turkey,United Arab Emirates,Uganda,Ukraine,Venezuela,Vietnam,Zambia,Zimbabwe,Sudan
UT (Unique WOS ID),,,,,,,,,,,,,,,,,,,,,
WOS:000904661800001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data_prep = pd.read_excel(f'D:\MARBURG VIRUS DISEASES/data/WOSCC/woscc_bon_23_02_verif_on_wos.xls')
data_prep = data_prep.rename(columns={"UT (Unique WOS ID)": "wos_ID"})
data_prep = data_prep.applymap(lambda s:s.lower() if type(s) == str else s)
#data_prep = data_prep.drop_duplicates('wos_ID')
data_prep.head(1)

,Publication Type,Authors,Book Authors,Book Editors,Book Group Authors,Author Full Names,Book Author Full Names,Group Authors,Article Title,Source Title,...,Web of Science Index,Research Areas,IDS Number,Pubmed Id,Open Access Designations,Highly Cited Status,Hot Paper Status,Date of Export,wos_ID,Web of Science Record
0,j,"bi, jh; wang, hj; han, qx; pei, hy; wang, hl; ...",NaN,NaN,NaN,"bi, jinhao; wang, haojie; han, qiuxue; pei, ho...",NaN,NaN,a rabies virus-vectored vaccine expressing two...,emerging microbes & infections,...,science citation index expanded (sci-expanded),immunology; infectious diseases; microbiology,7j5zq,36453198.0,"green accepted, gold",NaN,NaN,2023-02-22,wos:000904661800001,0


# Extract number of author

## preprocessing data

In [ ]:
# number of authors
data_prep['number_of_author'] =  pd.DataFrame(data_prep['Author Full Names'].str.split(';', expand=True)).count(axis='columns')
number_author = data_prep[['wos_ID','number_of_author']]
# number_author = number_author.reset_index().rename(columns = {'UT (Unique WOS ID)':'wos_ID'})
number_author['wos_ID'] = number_author['wos_ID'].str.lower()
number_author

C:\Users\dmbog\AppData\Local\Temp\ipykernel_14416\3455961627.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  number_author['wos_ID'] = number_author['wos_ID'].str.lower()


,wos_ID,number_of_author
0,wos:000904661800001,13
1,wos:000921279800001,11
2,wos:000909001200001,1
3,wos:000777045900001,14
4,wos:000849602200001,6
...,...,...
927,wos:a1968b358900049,2
928,wos:a1968a855300001,5
929,wos:a1968a855300006,3
930,wos:a1968a855300008,3


In [ ]:
number_author.describe()

,number_of_author
count,932.000000
mean,7.520386
std,6.188640
min,1.000000
25%,3.000000
50%,6.000000
75%,10.000000
max,76.000000


In [ ]:
number_author.isna().sum()

wos_ID              0
number_of_author    0
dtype: int64

In [ ]:
# number_author.head(60)

In [ ]:
data_and_number_author = pd.concat([data.set_index('wos_ID'),
                           number_author.set_index('wos_ID')],
                          axis=1)
data_and_number_author.head(2)

,Unnamed: 0,funding_yes,open_access_yes,Funding Name Preferred,europe and central asia,north america,africa,sub-saharan africa,east asia and pacific,/,...,world bank - multilateral,china association for science and technology - china,zhejiang university - china,Funding Orgs,Open Access Designations,"Times Cited, All Databases",180 Day Usage Count,Since 2013 Usage Count,Publication Year,number_of_author
wos_ID,,,,,,,,,,,,,,,,,,,,,
wos:000921279800001,1.0,1.0,1.0,National Institutes of Health (NIH)(United Sta...,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,national institutes of health (nih) [u19ai1427...,hybrid,NaN,NaN,NaN,2023.0,11
wos:000909001200001,2.0,1.0,1.0,Russian Science Foundation(Russian Science Fou...,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,russian science foundation; [22-13-00036],"gold, green accepted",NaN,3.0,3.0,2023.0,1


In [ ]:
# to drop rows with 0 values in column 'number_of_country':
data_and_number_author = data_and_number_author[data_and_number_author['number_of_author'] != 0]

In [ ]:
# data_and_number_author.describe()

In [ ]:
data_and_number_author.isna().sum()

Unnamed: 0                    355
funding_yes                   355
open_access_yes               355
Funding Name Preferred        357
europe and central asia       807
                             ... 
Times Cited, All Databases    385
180 Day Usage Count           696
Since 2013 Usage Count        372
Publication Year              359
number_of_author                0
Length: 521, dtype: int64

## analysis based on number of country

### proportion all

In [ ]:
data_and_number_author['number_of_author'].isna().sum()

0

In [ ]:
data_and_number_authorT = data_and_number_author.T

data_and_number_authorT['countries'] = data_and_number_authorT.index

number_of_author_mean_list = []
number_of_author_count_list = []
for country in data_and_number_author.columns:
    number_of_author_count_list.append(data_and_number_author.dropna(subset=[country])['number_of_author'].count())
    number_of_author_mean_list.append(data_and_number_author.dropna(subset=[country])['number_of_author'].mean())

# rename countries names on columns
countries = data_and_number_author.columns.to_list()
number_of_author_count  = number_of_author_count_list
number_of_author_mean = number_of_author_mean_list

data_and_number_authorT_count = pd.DataFrame({'countries': countries, 'all_number_of_author_count': number_of_author_count})
data_and_number_authorT_mean = pd.DataFrame({'countries': countries, 'all_number_of_author_mean': number_of_author_mean})

# ********
data_and_number_authorT_concat = data_and_number_authorT.reset_index().merge(data_and_number_authorT_count, on = 'countries', how='left')
data_and_number_authorT_concat = data_and_number_authorT_concat.merge(data_and_number_authorT_mean, on = 'countries', how='left')

data_and_number_authorT_concat_all = data_and_number_authorT_concat[['countries','all_number_of_author_count','all_number_of_author_mean']]

# save data
data_and_number_authorT_concat_all.to_csv(f'../TABLES/region_countries_org_number_of_author_count_mean.csv')
data_and_number_authorT_concat_all

,countries,all_number_of_author_count,all_number_of_author_mean
0,Unnamed: 0,577,9.091854
1,funding_yes,577,9.091854
2,open_access_yes,577,9.091854
3,Funding Name Preferred,575,9.102609
4,europe and central asia,125,8.776000
...,...,...,...
516,"Times Cited, All Databases",547,9.106033
517,180 Day Usage Count,236,10.093220
518,Since 2013 Usage Count,560,9.094643
519,Publication Year,573,9.101222


In [ ]:
len(number_of_author_count)

521

In [ ]:
len(number_of_author_mean)

521

In [ ]:
len(countries)

521

In [ ]:
vvvv

NameError: name 'vvvv' is not defined

### proportion fund

In [ ]:
data_and_number_author_fund = data_and_number_author.loc[data_and_number_author['funding_yes'] == 1]
data_and_number_author_fund.shape

In [ ]:
data_and_number_authorT = data_and_number_author_fund.T

data_and_number_authorT['countries'] = data_and_number_authorT.index

number_of_author_mean_list = []
number_of_author_count_list = []
for country in data_and_number_author_fund.columns:
    number_of_author_count_list.append(data_and_number_author_fund.dropna(subset=[country])['number_of_author'].count())
    number_of_author_mean_list.append(data_and_number_author_fund.dropna(subset=[country])['number_of_author'].mean())

# rename countries names on columns
countries = data_and_number_author_fund.columns.to_list()
number_of_author_count  = number_of_author_count_list
number_of_author_mean = number_of_author_mean_list

data_and_number_authorT_count = pd.DataFrame({'countries': countries, 'fund_number_of_author_count': number_of_author_count})
data_and_number_authorT_mean = pd.DataFrame({'countries': countries, 'fund_number_of_author_mean': number_of_author_mean})

# ********
data_and_number_authorT_concat = data_and_number_authorT.reset_index().merge(data_and_number_authorT_count, on = 'countries', how='left')
data_and_number_authorT_concat = data_and_number_authorT_concat.merge(data_and_number_authorT_mean, on = 'countries', how='left')

data_and_number_authorT_concat_number_of_author_fund = data_and_number_authorT_concat[['countries','fund_number_of_author_count','fund_number_of_author_mean']]

# save data
# data_and_number_authorT_concat_all.to_csv(f'../TABLES/countries_fund_number_of_author_count_mean.csv')
data_and_number_authorT_concat_number_of_author_fund

### proportion no fund

In [ ]:
data_and_number_author_no_fund = data_and_number_author.loc[data_and_number_author['funding_yes'] == 0]
data_and_number_author_no_fund.shape

In [ ]:
data_and_number_authorT = data_and_number_author_no_fund.T

data_and_number_authorT['countries'] = data_and_number_authorT.index

number_of_author_mean_list = []
number_of_author_count_list = []
for country in data_and_number_author_no_fund.columns:
    number_of_author_count_list.append(data_and_number_author_no_fund.dropna(subset=[country])['number_of_author'].count())
    number_of_author_mean_list.append(data_and_number_author_no_fund.dropna(subset=[country])['number_of_author'].mean())

# rename countries names on columns
countries = data_and_number_author_no_fund.columns.to_list()
number_of_author_count  = number_of_author_count_list
number_of_author_mean = number_of_author_mean_list

data_and_number_authorT_count = pd.DataFrame({'countries': countries, 'no_fund_number_of_author_count': number_of_author_count})
data_and_number_authorT_mean = pd.DataFrame({'countries': countries, 'no_fund_number_of_author_mean': number_of_author_mean})

# ********
data_and_number_authorT_concat = data_and_number_authorT.reset_index().merge(data_and_number_authorT_count, on = 'countries', how='left')
data_and_number_authorT_concat = data_and_number_authorT_concat.merge(data_and_number_authorT_mean, on = 'countries', how='left')

data_and_number_authorT_concat_number_of_author_nofund = data_and_number_authorT_concat[['countries','no_fund_number_of_author_count','no_fund_number_of_author_mean']]

# save data
# data_and_number_authorT_concat_all.to_csv(f'../TABLES/countries_no_fund_number_of_author_count_mean.csv')
data_and_number_authorT_concat_number_of_author_nofund

### concat and save data

In [ ]:
# concatenate dataframes on index
concat_data_number_of_author_list = pd.concat([data_and_number_authorT_concat_all.set_index('countries'),
                                    data_and_number_authorT_concat_number_of_author_fund.set_index('countries'),
                                    data_and_number_authorT_concat_number_of_author_nofund.set_index('countries')],
                                    axis=1)
# save data
concat_data_number_of_author_list.to_csv(f'../TABLES/concat_data_number_of_author_list.csv')
concat_data_number_of_author_list

### significance

In [ ]:
number_author.describe()

In [ ]:
fund_number_of_author = data_and_number_author.loc[data_and_number_author['funding_yes'] == 1]['number_of_author']#.to_list()
fund_number_of_author[:10]

In [ ]:
fund_number_of_author.describe()

In [ ]:
no_fund_number_of_author = data_and_number_author.loc[data_and_number_author['funding_yes'] == 0]['number_of_author']#.to_list()

In [ ]:
no_fund_number_of_author.describe()

In [ ]:
test_sig(fund_number_of_author, no_fund_number_of_author)

In [ ]:
fund_number_of_author_new = [x for x in fund_number_of_author if x != 0]
no_fund_number_of_author_new = [x for x in no_fund_number_of_author if x != 0]

In [ ]:
test_sig(fund_number_of_author_new, no_fund_number_of_author_new)

In [ ]:
import plotly.express as px
fig = px.box(data_and_number_author, x="funding_yes", y='number_of_author', color="funding_yes")
# setting the y-axis range from 0 to 50
fig.update_layout(yaxis_range=[-2,100])
fig.update_layout(font_size=20)

fig.show()